# IRH v21.0: Interactive RG Flow Explorer

**THEORETICAL FOUNDATION**: IRH21.md §1.2-1.3

This notebook provides interactive exploration of the renormalization group flow
from UV to the Cosmic Fixed Point.

## Contents

1. Wetterich equation integration
2. β-function visualization
3. Flow trajectories in coupling space
4. Fixed point stability analysis
5. Spectral dimension flow

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Fixed-point values (Eq. 1.14)
LAMBDA_STAR = 48 * np.pi**2 / 9
GAMMA_STAR = 32 * np.pi**2 / 3
MU_STAR = 16 * np.pi**2

print(f"Cosmic Fixed Point:")
print(f"  λ̃* = {LAMBDA_STAR:.6f}")
print(f"  γ̃* = {GAMMA_STAR:.6f}")
print(f"  μ̃* = {MU_STAR:.6f}")